# *Adversarial validation*

Muitas vezes, treinamos um modelo com bom desempenho na predição do conjunto de validação (ou com boas métricas de validação cruzada), mas que não apresenta bons resultados na predição do conjunto de teste ou de dados futuros. Isso pode ocorrer em processos que mudam com o tempo, fazendo com que novos dados sejam muito diferentes dos dados usados para treinar os modelos.

Uma forma de verificar se os seus dados de teste são muito diferentes dos dados de treinamento é utilizando o método *Adversarial validation* (não encontrei tradução, algo como validação adversarial). Apesar do nome complicado, o método é simples, e consiste nas seguintes etapas:

- Ignore o valor da sua variável **y**, e crie uma nova variável binária: o valor **0** será atribuído ao conjunto de treinamento, e o valor **1**, ao conjunto de teste
- Junte os dados de treinamento e teste, e crie um modelo de classificação. Você pode usar validação cruzada para calcular as métricas desse modelo
- Avalie as métricas de qualidade, em especial, o valor da área sob a curva ROC

Qual é o resultado esperado? Digamos que seus conjuntos de treinamento e teste sejam parecidos (pertencem à mesma população). Nesse caso, o modelo de classificação terá péssimos valores de métricas de qualidade, pois será incapaz de detectar diferenças significativas entre as "duas classes". O valor da área sob a curva ROC será próximo a 0.5.

Porém, é possível que seu conjunto de teste seja, de fato, muito diferente do conjunto de treinamento. Nesse caso, o modelo binário conseguirá facilmente separar as duas classes, e o valor da área sob a curva ROC será próximo a 1.0.

Vejamos como aplicar esse método:

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Delaney_descriptors.csv", sep=";")
df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Solubilidade_medida
0,10.253329,-1.701605,10.253329,0.486602,0.217518,457.432,430.216,457.158411,178,0,...,0,0,0,0,0,0,0,0,0,-0.77
1,11.724911,-0.145880,11.724911,0.145880,0.811283,201.225,190.137,201.078979,76,0,...,0,0,0,0,0,0,0,0,0,-3.30
2,10.020498,0.845090,10.020498,0.845090,0.343706,152.237,136.109,152.120115,62,0,...,0,0,0,0,0,0,0,0,0,-2.06
3,2.270278,1.301055,2.270278,1.301055,0.291526,278.354,264.242,278.109550,102,0,...,0,0,0,0,0,0,0,0,0,-7.87
4,2.041667,1.712963,2.041667,1.712963,0.448927,84.143,80.111,84.003371,26,0,...,0,0,0,0,0,0,1,0,0,-1.33


In [3]:
# Vamos selecionar apenas algumas colunas
X = df[["MolWt", "FractionCSP3", "MolLogP", "NumAromaticRings", "NumHAcceptors", 
        "NumHDonors", "NumRotatableBonds", "TPSA"]]
y = df.iloc[:, -1]  # resposta

In [4]:
# Separando um conjunto de treinamento e um de teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [5]:
# Vamos criar novas classes fictícias para os valores de treinamento e teste
# Os valores de y_train recebem a classe 0, e os de y_test, 1
y_train_adv = [0]*(len(y_train))
y_test_adv = [1]*(len(y_test))
y_adv = y_train_adv + y_test_adv
len(y_adv)

1128

In [6]:
# Agrupando os dados novamente
X_adv = pd.concat([X_train, X_test])
X_adv.shape

(1128, 8)

In [10]:
# Vamos treinar um modelo de classificação e availá-lo usando validação cruzada
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=0)
scores = cross_val_score(clf, X_adv, y_adv, cv=5, scoring='roc_auc')
scores

array([0.41318277, 0.415395  , 0.45785321, 0.48298817, 0.48837701])

Um valor de curva ROC próximo a 1.0 indica um bom classificador, enquanto valores perto de 0.5 indicam classificadores ruins. Vemos nesse caso que a *adversarial validation* indica que nossos dados de teste não são significativamente distintos dos de treinamento.